## Behaviral Law Engine
- Stage: Cambrian
- Version: Pomoria

In [1]:
import logging 
logging.basicConfig(level=logging.CRITICAL)

In [2]:
import os
import itertools
import time
import copy
import random
import pickle
import numpy as np

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# from tensorboardX import SummaryWriter

use_cuda = torch.cuda.is_available()
# use_cuda = False
print("use_cuda: {}".format(use_cuda))

use_cuda: True


In [4]:
from MorpheusInterpreter import *
from ProgramSpace import *

In [5]:
torch.__version__

'1.0.0'

In [6]:
class ProgramDataset(Dataset):
    def __init__(self, p_config=None, pn_program=None, p_generator=None, p_sourceps=None):
        self.config = p_config
        self.n_program = pn_program
        self.generator = p_generator
        self.source_ps = p_sourceps
        
        self.shell_list = self.source_ps.get_neighboring_shells()
        self.shell_dict = {
            self.shell_list[i]:i for i in range(len(self.shell_list))
        }
        
    def __len__(self):
        return self.n_program
    
    '''
    shortened method: only valid for size 1 programs
    '''
    def __getitem__(self, p_ind):
        # d_solution = self.generator.get_new_chain_program(2)
        d_solution = None
        while d_solution is None:
            d_solution = self.generator.get_new_size1_program()
        d_shell = d_solution.shells[0]
        dind_shell = self.shell_dict[d_shell]
        dabs_input = d_solution.interpreter.camb_get_ventogyrus(d_solution.inputs[0])
        return (np.array(dind_shell), np.array(dabs_input))

In [7]:
class BLE(nn.Module):
    def __init__(self, p_config=None):
        super(BLE,self).__init__()
        self.config = p_config
        self.fc1 = nn.Linear(
            self.config["BLE"]["input_size"],
            2048,
        )
        self.fc2 = nn.Linear(
            2048,
            self.config["BLE"]["output_size"],
        )
    def forward(self, p_abs):
        # p_abs: (B, 15*7+1)
        out1 = F.relu(self.fc1(p_abs))
        out2 = torch.log_softmax(self.fc2(out1),dim=1)
        return out2
        

In [8]:
def BLETrain(p_config, p_engine, pld_data, p_optim, p_lossfn):
    for d_ep in range(p_config["BLE"]["n_epoch"]):
        ep_loss_list = []
        for batch_idx, (b_ind, b_input) in enumerate(pld_data):
            p_engine.train()
            if use_cuda:
                tb_ind = Variable(b_ind).long().cuda()
                tb_input = Variable(b_input).float().cuda()
            else:
                tb_ind = Variable(b_ind).long()
                tb_input = Variable(b_input).float()
            
            tb_preds = p_engine(tb_input) # (B, output_size)
            tb_loss = p_lossfn(tb_preds, tb_ind)
            ep_loss_list.append(tb_loss)
            p_optim.zero_grad()
            tb_loss.backward()
            p_optim.step()
            
            print("\r# Train Ep:{}, B:{}, epLoss:{:.2f}".format(
                d_ep, batch_idx, sum(ep_loss_list)/len(ep_loss_list)
            ),end="")
        print()
            

In [9]:
def BLEManualTest(p_engine, p_sourceps):
    p_engine.eval()
    p_shell_list = p_sourceps.get_neighboring_shells()
    output_id = -1
    while output_id<150:
        output_id += 1
        r_table = p_sourceps.interpreter.random_table()
#         p_sourceps.interpreter.print_obj(r_table)
        r_abs = p_sourceps.interpreter.camb_get_ventogyrus(r_table)
        if use_cuda:
            tr_abs = Variable(torch.tensor([r_abs],dtype=torch.float)).cuda()
        else:
            tr_abs = Variable(torch.tensor([r_abs],dtype=torch.float))
        tr_preds = p_engine(tr_abs)
        r_preds = tr_preds.data.cpu().numpy().flatten()
        arg_preds = np.argsort(r_preds)[::-1]
        # then visualize them out
        f = open("./outputs/0811BLE/0811BLE_input_{}.txt".format(output_id),"w")
        f.write(str(p_sourceps.interpreter.renv(r_table)))
        f.write("\n")
        for i in range(len(arg_preds)):
            did_shell = p_shell_list[arg_preds[i]]
            d_name = p_sourceps.prod_list[did_shell[0]].name
            d_param = []
            for pp in did_shell[1]:
                d_param.append(str(p_sourceps.node_list[pp]))
            d_ps = p_sourceps.make_copy()
            d_ps.inputs = [r_table]
            d_sts = d_ps.add_neighboring_shell(p_shell_list[arg_preds[i]])
            if d_sts:
                d_ps.output = d_ps.node_list[-1].ps_data
                d_snty= d_ps.interpreter.sanity_check(d_ps)
            else:
                d_snty= (False,None)
#             print("({}/{})#{}, {}, {}".format(
#                 "✓" if d_sts else "x",
#                 "✓" if d_snty[0] else "x",
#                 i, d_name, ",".join(d_param)
#             ))
            # output to file
            f.write("({}/{}/{:.4f})#{}, {}, {}\n".format(
                "✓" if d_sts else "x",
                "✓" if d_snty[0] else "x",
                r_preds[arg_preds[i]],
                i, d_name, ",".join(d_param)
            ))
            
        f.close()
#         input("======== PAUSE ========")

In [10]:
m_interpreter = MorpheusInterpreter()
m_spec = S.parse_file('./example/camb6.tyrell')
m_generator = MorpheusGenerator(m_spec, m_interpreter)
m_ps = ProgramSpace(
    m_spec, m_interpreter, [None], None
)

In [11]:
m_config = {
    "BLE":{
        "ntrain_program":1600,
        "input_size": 15*7+1,
        "output_size": len(m_ps.get_neighboring_shells()),
        "n_epoch": 80,
    }
}

In [12]:
dt_pd = ProgramDataset(
    p_config=m_config, 
    pn_program=m_config["BLE"]["ntrain_program"], 
    p_generator=m_generator, 
    p_sourceps=m_ps
)
ld_pd = DataLoader(dataset=dt_pd, batch_size=16, shuffle=True)

In [13]:
ble = BLE(p_config=m_config)
m_lossfn = nn.NLLLoss()
m_optim = torch.optim.Adam(ble.parameters())
if use_cuda:
    ble = ble.cuda()
    m_lossfn = m_lossfn.cuda()

In [14]:
BLETrain(m_config, ble, ld_pd, m_optim, m_lossfn)

# Train Ep:0, B:99, epLoss:5.10
# Train Ep:1, B:99, epLoss:4.55
# Train Ep:2, B:99, epLoss:4.27
# Train Ep:3, B:99, epLoss:4.19
# Train Ep:4, B:99, epLoss:4.11
# Train Ep:5, B:99, epLoss:4.07
# Train Ep:6, B:99, epLoss:4.02
# Train Ep:7, B:99, epLoss:4.03
# Train Ep:8, B:99, epLoss:3.96
# Train Ep:9, B:99, epLoss:3.95
# Train Ep:10, B:99, epLoss:3.99
# Train Ep:11, B:99, epLoss:3.91
# Train Ep:12, B:99, epLoss:3.99
# Train Ep:13, B:99, epLoss:3.87
# Train Ep:14, B:99, epLoss:3.89
# Train Ep:15, B:99, epLoss:3.88
# Train Ep:16, B:99, epLoss:3.91
# Train Ep:17, B:99, epLoss:3.91
# Train Ep:18, B:99, epLoss:3.85
# Train Ep:19, B:99, epLoss:3.82
# Train Ep:20, B:99, epLoss:3.85
# Train Ep:21, B:99, epLoss:3.84
# Train Ep:22, B:99, epLoss:3.88
# Train Ep:23, B:99, epLoss:3.80
# Train Ep:24, B:99, epLoss:3.89
# Train Ep:25, B:99, epLoss:3.75
# Train Ep:26, B:99, epLoss:3.82
# Train Ep:27, B:99, epLoss:3.75
# Train Ep:28, B:99, epLoss:3.79
# Train Ep:29, B:99, epLoss:3.85
# Train Ep:30, B:99,

In [15]:
BLEManualTest(ble, m_ps)

In [17]:
torch.save(ble.state_dict(),"./saved_models/0811CambBLE_Newton_ep80.pt")